In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch import nn
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')
    

In [2]:
import attr

In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [4]:
# context initialization
with initialize(config_path="../training/conf/network/", job_name="test_app"):
    cfg = compose(config_name="conv_transformer")
    print(OmegaConf.to_yaml(cfg))
    print(cfg)

encoder:
  _target_: text_recognizer.networks.encoders.efficientnet.EfficientNet
  arch: b0
  out_channels: 1280
  stochastic_dropout_rate: 0.2
  bn_momentum: 0.99
  bn_eps: 0.001
decoder:
  _target_: text_recognizer.networks.transformer.Decoder
  dim: 256
  depth: 2
  num_heads: 8
  attn_fn: text_recognizer.networks.transformer.attention.Attention
  attn_kwargs:
    num_heads: 8
    dim_head: 64
    dropout_rate: 0.2
  norm_fn: torch.nn.LayerNorm
  ff_fn: text_recognizer.networks.transformer.mlp.FeedForward
  ff_kwargs:
    dim: 256
    dim_out: null
    expansion_factor: 4
    glu: true
    dropout_rate: 0.2
  rotary_emb: null
  rotary_emb_dim: null
  cross_attend: true
  pre_norm: true
_target_: text_recognizer.networks.conv_transformer.ConvTransformer
input_dims:
- 1
- 576
- 640
hidden_dim: 256
dropout_rate: 0.2
max_output_len: 682
num_classes: 1004
start_token: <s>
end_token: <e>
pad_token: <p>

{'encoder': {'_target_': 'text_recognizer.networks.encoders.efficientnet.EfficientNet'

In [5]:
with initialize(config_path="../training/conf/mapping/", job_name="test_app"):
    cfg = compose(config_name="word_piece")
    print(OmegaConf.to_yaml(cfg))
    print(cfg)

_target_: text_recognizer.data.mappings.WordPieceMapping
num_features: 1000
tokens: iamdb_1kwp_tokens_1000.txt
lexicon: iamdb_1kwp_lex_1000.txt
data_dir: null
use_words: false
prepend_wordsep: false
special_tokens:
- <s>
- <e>
- <p>
extra_symbols:
- '

  '

{'_target_': 'text_recognizer.data.mappings.WordPieceMapping', 'num_features': 1000, 'tokens': 'iamdb_1kwp_tokens_1000.txt', 'lexicon': 'iamdb_1kwp_lex_1000.txt', 'data_dir': None, 'use_words': False, 'prepend_wordsep': False, 'special_tokens': ['<s>', '<e>', '<p>'], 'extra_symbols': ['\n']}


In [6]:
cfg

{'_target_': 'text_recognizer.data.mappings.WordPieceMapping', 'num_features': 1000, 'tokens': 'iamdb_1kwp_tokens_1000.txt', 'lexicon': 'iamdb_1kwp_lex_1000.txt', 'data_dir': None, 'use_words': False, 'prepend_wordsep': False, 'special_tokens': ['<s>', '<e>', '<p>'], 'extra_symbols': ['\n']}

In [8]:
mapping = instantiate(cfg)

2021-07-29 23:02:56.650 | DEBUG    | text_recognizer.data.mappings:_configure_wordpiece_processor:104 - Using data dir: /home/aktersnurra/projects/text-recognizer/data/downloaded/iam/iamdb


In [9]:
mapping.get_index("<p>")

tensor([1002])

In [10]:
len(mapping)

1006

In [15]:
mapping.get_index("\n")

tensor([1003])